In [1]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt

In [75]:
# %load ../Tools/Histogram.py
# %load ../Tools/Histogram.py
import numpy as np


class Histogram():
    '''
    Class to storage histograms
    '''
    def __init__(self, data=[], nbin=None, minlim=None, maxlim=None):
        '''
        Creates an histogram from a given a list (empty by defoult),
        with low and high limits, by defoult are the minimum and maximum
        of de data sample and the bin number, by default the number of
        data squareroot
        '''

        if len(data) == 0:
            if minlim == 'nada' or maxlim == 'nada' or nbin == 'nada':
                auxs = 'In order to build an empty histogram it '
                auxs += 'is needed a minlim, maxlim and nbin'
                raise ValueError(auxs)
                return
            else:
                self.minlim = minlim
                self.maxlim = maxlim
                self.nbin = nbin

        else:
            if minlim is None:
                self.minlim = min(data)
            else:
                self.minlim = minlim
            if maxlim is None:
                self.maxlim = max(data)
            else:
                self.maxlim = maxlim
            if nbin is None:
                self.nbin = int(len(data)**0.5)
            else:
                self.nbin = nbin

        self.binsize = float(self.maxlim-self.minlim)/float(self.nbin)
        self.hist = np.array([])
        self.bins = np.array([])

        self.Build_hist(data)

    def __add__(self, hist):
        '''
        Suming histograms
        '''

        if not self.minlim == hist.minlim or not self.maxlim == hist.maxlim\
                or not self.nbin == hist.nbin:
            raise ValueError('Histograms are not compatible')
        else:
            auxhist = Histogram([], self.nbin, self.minlim, self.maxlim)
            auxhist.hist = self.hist + hist.hist
        return auxhist

    def Build_hist(self, data):
        '''
        Creating histogram using numpy ones but with bin centers instead
        '''
        auxhist, auxbins = np.histogram(np.array(data), self.nbin,
                                        [self.minlim, self.maxlim])
        auxbins = auxbins[:-1] + np.diff(auxbins)/2.
        self.hist = auxhist
        self.bins = auxbins

    def Fill_hist(self, data):
        '''
        Filling histogram
        '''
        data = np.array(data)
        self.hist += np.histogram(np.array(data), self.nbin, [self.minlim,
                                                              self.maxlim])[0]

    def Scale(self, factor):
        '''
        Explicit __rmul__, scales the histogram by a factor
        '''
        self.hist = factor*self.hist
        return self


In [24]:
auxlist = np.array([0,1,2,3,1,2,2,1,3,0,1,2,3]*100)
auxhist,auxbins = np.histogram(auxlist,3,[0,3])
%timeit auxhist,auxbins = np.histogram(auxlist,3,[0,3])
auxbincenter = [(auxbins[i]+auxbins[i+1])/2. for i in np.arange(len(auxbins)-1)]
print auxhist,auxbins[:-1],auxbincenter
%timeit Histogram(auxlist,3,0,3)
#print Build_hist(auxlist,3,0,3)

1000 loops, best of 3: 145 µs per loop
[200 400 700] [ 0.  1.  2.] [0.5, 1.5, 2.5]
10000 loops, best of 3: 148 µs per loop


In [76]:
a = Histogram(auxlist,3,0,3)
ap = Histogram(auxlist,3,1,3)
print a.hist

[200 400 700]


In [77]:
b = a+a.Scale(2.)
print b.hist

[  800.  1600.  2800.]


In [78]:
print type(a),type(a.Scale(1))

<type 'instance'> <type 'instance'>
